In [12]:
# %pip install keras==3.0.5
# %pip install tensorflow
# %pip install opencv-python
# %pip install matplotlib
# %pip install moviepy

In [13]:
from keras.models import load_model
import tensorflow as tf
import keras
import cv2
import os
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split

# from tensorflow.keras.layers import *
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.utils import plot_model
# %matplotlib inline

from moviepy.editor import *

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("OpenCV version:", cv2.__version__)

TensorFlow version: 2.16.1
Keras version: 3.0.5
OpenCV version: 4.9.0


In [14]:
def custom_load_model(model_path):
    custom_objects = {'batch_shape': tf.keras.layers.InputLayer}  # Define custom objects to handle batch_shape
    return tf.keras.models.load_model(model_path, custom_objects=custom_objects)

In [15]:
LRCN_model = tf.keras.models.load_model('violence_nonviolence.h5') #load the model

In [16]:
LRCN_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_12             │ (None, 20, 64, 64, 16) │           448 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_13             │ (None, 20, 16, 16, 16) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_14             │ (None, 20, 16, 16, 16) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_15             │ (None, 20, 16, 16, 32) │         4,640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_16             │ (None, 20, 4, 4, 32)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_17             │ (None, 20, 4, 4, 32)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_18             │ (None, 20, 4, 4, 64)   │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_19             │ (None, 20, 2, 2, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_20             │ (None, 20, 2, 2, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_21             │ (None, 20, 2, 2, 64)   │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_22             │ (None, 20, 1, 1, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_23             │ (None, 20, 64)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,996 (285.14 KB)

 Trainable params: 72,994 (285.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [17]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

# CLASSES_LIST = ["boxing", "handclapping", "handwaving", "jogging", "running", "walking"]
CLASSES_LIST = ["non-violence", "violence"]

In [18]:
# def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH):
#     '''
#     This function will perform action recognition on a video using the LRCN model.
#     Args:
#     video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
#     output_file_path: The path where the ouput video with the predicted action being performed overlayed will be stored.
#     SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
#     '''

#     # Initialize the VideoCapture object to read from the video file.
#     video_reader = cv2.VideoCapture(video_file_path)

#     # Get the width and height of the video.
#     original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
#     original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # Initialize the VideoWriter Object to store the output video in the disk.
#     video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'),
#                                    video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

#     # Declare a queue to store video frames.
#     frames_queue = deque(maxlen = SEQUENCE_LENGTH)

#     # Initialize a variable to store the predicted action being performed in the video.
#     predicted_class_name = ''

#     # Iterate until the video is accessed successfully.
#     while video_reader.isOpened():

#         # Read the frame.
#         ok, frame = video_reader.read()

#         # Check if frame is not read properly then break the loop.
#         if not ok:
#             break

#         # Resize the Frame to fixed Dimensions.
#         resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

#         # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
#         normalized_frame = resized_frame / 255

#         # Appending the pre-processed frame into the frames list.
#         frames_queue.append(normalized_frame)

#         # Check if the number of frames in the queue are equal to the fixed sequence length.
#         if len(frames_queue) == SEQUENCE_LENGTH:

#             # Pass the normalized frames to the model and get the predicted probabilities.
#             predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_queue, axis = 0))[0]

#             # Get the index of class with highest probability.
#             predicted_label = np.argmax(predicted_labels_probabilities)

#             # Get the class name using the retrieved index.
#             predicted_class_name = CLASSES_LIST[predicted_label]

#         # Write predicted class name on top of the frame.
#         cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#         # Write The frame into the disk using the VideoWriter Object.
#         video_writer.write(frame)

#     # Release the VideoCapture and VideoWriter objects.
#     video_reader.release()
#     video_writer.release()

In [19]:
# output_video_file_path = 'output_video.mp4'

# # Perform Action Recognition on the Test Video.
# predict_on_video('input_video.mp4', output_video_file_path, SEQUENCE_LENGTH)

# # Display the output video.
# VideoFileClip(output_video_file_path, audio=False, target_resolution=(300,None)).ipython_display()

In [20]:
def predict_on_webcam():
    # Initialize the VideoCapture object to read from the webcam
    video_capture = cv2.VideoCapture(0)

    # Declare a queue to store video frames
    frames_queue = deque(maxlen=SEQUENCE_LENGTH)

    while True:
        # Read frame from webcam
        ret, frame = video_capture.read()

        # Resize the frame to fixed dimensions
        resized_frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))

        # Normalize the resized frame
        normalized_frame = resized_frame / 255.0

        # Append the pre-processed frame into the frames list
        frames_queue.append(normalized_frame)

        # Check if the number of frames in the queue are equal to the fixed sequence length
        if len(frames_queue) == SEQUENCE_LENGTH:
            # Pass the normalized frames to the model and get the predicted probabilities
            predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_queue, axis=0))[0]

            # Get the probability of violence
            violence_probability = predicted_labels_probabilities[CLASSES_LIST.index('non-violence')]

            # Check if the probability of violence exceeds 90%
            if violence_probability >= 0.9:
                predicted_class_name = 'violence'
            else:
                predicted_class_name = 'non-violence'

            # Overlay predicted class name on top of the frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame with predicted label
        cv2.imshow('Video', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object and close the OpenCV window
    video_capture.release()
    cv2.destroyAllWindows()


In [21]:
# import time
# import csv

# def predict_on_webcam():
#     # Initialize the VideoCapture object to read from the webcam
#     video_capture = cv2.VideoCapture(0)

#     # Declare a queue to store video frames
#     frames_queue = deque(maxlen=SEQUENCE_LENGTH)

#     # Variables to track violence detection duration
#     violence_start_time = None
#     violence_end_time = None
#     violence_detected = False

#     while True:
#         # Read frame from webcam
#         ret, frame = video_capture.read()

#         # Convert frame to 8-bit unsigned integer
#         frame = cv2.convertScaleAbs(frame)

#         # Resize the frame to fixed dimensions
#         resized_frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))

#         # Normalize the resized frame
#         normalized_frame = resized_frame / 255.0

#         # Append the pre-processed frame into the frames queue
#         frames_queue.append(normalized_frame)

#         # Check if the number of frames in the queue are equal to the fixed sequence length
#         if len(frames_queue) == SEQUENCE_LENGTH:
#             # Pass the normalized frames to the model and get the predicted probabilities
#             predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_queue, axis=0))[0]

#             # Get the probability of violence
#             violence_probability = predicted_labels_probabilities[CLASSES_LIST.index('violence')]

#             # Check if violence is detected
#             if violence_probability >= 0.9:
#                 if not violence_detected:
#                     # Record start time of violence detection
#                     violence_start_time = time.time()
#                     violence_detected = True
#                 else:
#                     # Update end time of violence detection
#                     violence_end_time = time.time()

#                 # If violence has been detected for more than 5 seconds, save the clip and times
#                 if violence_start_time is not None and violence_end_time is not None:
#                     if violence_end_time - violence_start_time >= 5:
#                         # Save the clip
#                         save_video_clip(frames_queue)

#                         # Save the start and end times of violence detection to CSV file
#                         save_violence_times(violence_start_time, violence_end_time)

#             else:
#                 # Reset violence detection variables
#                 violence_start_time = None
#                 violence_end_time = None
#                 violence_detected = False

#             # Overlay real-time detection label on the frame
#             detection_label = 'Violence Detected' if violence_detected else 'No Violence Detected'
#             cv2.putText(frame, detection_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

#         # Display the frame with predicted label
#         cv2.imshow('Video', frame)

#         # Break the loop if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     # Release the VideoCapture object and close the OpenCV window
#     video_capture.release()
#     cv2.destroyAllWindows()


# def save_video_clip(frames_queue):
#     # Concatenate frames in the queue to create the video clip
#     video_clip = np.array(frames_queue)
    
#     # Write the video clip to a file
#     # Replace 'output_clip.mp4' with desired filename
#     # Use appropriate codec and frame rate as per your requirement
#     out = cv2.VideoWriter('output_clip.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (IMAGE_WIDTH, IMAGE_HEIGHT))
#     for frame in video_clip:
#         out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
#     out.release()

# def save_violence_times(start_time, end_time):
#     # Write the start and end times to a CSV file
#     with open('violence_times.csv', mode='a') as file:
#         writer = csv.writer(file)
#         writer.writerow([start_time, end_time])

# # Call the function to start violence prediction on webcam
# predict_on_webcam()


In [22]:
# Perform action recognition on webcam stream
predict_on_webcam()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━